<a href="https://colab.research.google.com/github/kimbabramun/test/blob/master/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!ls "/gdrive/My Drive/Colab Notebooks/data2"


ham  spam


In [0]:
import numpy as np
import cv2
import math
import os
import csv
from PIL import Image, ImageFile
from skimage.color import rgb2gray
from multiprocessing import Pool
from skimage.feature import local_binary_pattern # Local Binary Pattern function
from scipy.stats import itemfreq # To calculate a normalized histogram
import scipy.stats as sp
from skimage.feature import hog
from scipy.ndimage.measurements import label
from scipy import signal as sg
import imageio
import time # tocheck time for feature extraction


ImageFile.LOAD_TRUNCATED_IMAGES = True

#############################################################################
##                                                                         ##
##                                                                         ##
## Which : Compression Ratio (Metadata domain)
## What  : Ratio of Size of the original image to the changed image
## Why   : Spammers might try to compress image as far as possible.
##         Thus could show difference between spam and ham images
##         Spam image generally has high compression rate
## How   : (height * width * bitdepth) / Size of the image
##
##                                                                         ##
##                                                                         ##
#############################################################################
def calc_compression_ratio(img,image,file):
    height, width ,channels= img.shape
    file_size = os.stat(path1 +'/'+folder+'/'+file).st_size
    depth = image.bits * channels
    compression_ratio = (height * width * depth )/file_size
    return compression_ratio


#############################################################################
##                                                                         ##
##                                                                         ##
## Which : Aspect Ratio (Metadata domain)
## What  : Ratio of Size of the original image to the changed image
## Why   : Spammers might try to compress image as far as possible.
##         Thus could show difference between spam and ham images
##         Spam image generally has high compression rate
## How   : (height * width * bitdepth) / Size of the image
##
##                                                                         ##
##                                                                         ##
#############################################################################
def calc_aspect_ratio(img):
    height, width ,channels= img.shape
    aspect_ratio = width/height
    return aspect_ratio




#############################################################################
##                                                                         ##
##                                                                         ##
## Which : Number of Edge pixels(Shape domain)
## What  : Edges is the points in a image at which image brightness
##         changes sharpely or has discontinuities and are organized
##         into a set of curved line segments
## Why   : Spam images are combinations of text and images, they are
##         likely to have more number of edges compared to genuine images
## How   : Identifying the edges using Canny edge detection
##         method and counting the number of edges
##
##                                                                         ##
##                                                                         ##
#############################################################################
def calc_edge_count(img):
    edges = cv2.Canny(img,100,200)
    edge_count = np.count_nonzero(edges)
    return edge_count


#############################################################################
##                                                                         ##
##                                                                         ##
## Which : Average length of edges(Shape domain)
## What  : Edges is the points in a image at which image brightness
##         changes sharpely or has discontinuities and are organized
##         into a set of curved line segments
## Why   : Spam images are combinations of text and images, they are
##         likely to have more number of edges compared to genuine images
## How   : Identifying the edges using Canny edge detection
##         method and counting the number of edges
##
##                                                                         ##
##                                                                         ##
#############################################################################
def calc_avg_edge_length(img):
    edges = cv2.Canny(img,100,200)
    length = sum(map(sum, edges))
    num_features = np.count_nonzero(edges == 255)
    avg_edge_len= length/num_features
    return avg_edge_len






#############################################################################
##                                                                         ##
##                                                                         ##
## Which : Signal to Noise Ratio (SNR)(Noise domain)
## What  : SNR is used as a measure of the sensitivity of a image.
## Why   : Genuine images have significant noise components
## How   : Ratio of Mean to the standard deviation
##
##                                                                         ##
##                                                                         ##
#############################################################################
def signaltonoise(a, axis=0, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m/sd)



def calc_snr(img):
    snr = signaltonoise(img, axis=None)
    return snr

#############################################################################
##                                                                         ##
##                                                                         ##
## Which : Entropy of Noise(Noise domain)
## What  : The LBP contained in the image reveals the similarity of
##         differences between the neighboring pixels and can be used
##         to analyse texture information
## Why   : Genuine images are likely to have more information contained
##         in the LBP histogram.Spam images are likely to have more
##         smoother background and foreground features
## How   :
##
##                                                                         ##
##                                                                         ##
#############################################################################
def calc_entropy_noise(img):
    img_gray = rgb2gray(img)
    H, W = img_gray.shape

    M = [[1, -2, 1],
       [-2, 4, -2],
       [1, -2, 1]]

    sigma = np.sum(np.sum(np.absolute(sg.convolve2d(img_gray, M))))
    sigma = sigma * math.sqrt(0.5 * math.pi) / (6 * (W-2) * (H-2))

    return sigma



#############################################################################
##                                                                         ##
##                                                                         ##
## Which : Entropy of Local Binary Pattern Histogram(Texture domain)
## What  : The LBP contained in the image reveals the similarity of
##         differences between the neighboring pixels and can be used
##         to analyse texture information
## Why   : Genuine images are likely to have more information contained
##         in the LBP histogram.Spam images are likely to have more
##         smoother background and foreground features
## How   :
##
##                                                                         ##
##                                                                         ##
#############################################################################
def calc_entropy_lbp(img):
    img_gray = rgb2gray(img)
    radius = 3
    # Number of points to be considered as neighbours
    no_points = 8 * radius
    # Uniform LBP is used
    lbp = local_binary_pattern(img_gray, no_points, radius, method='uniform')
    # Calculate the histogram
    x = itemfreq(lbp.ravel())
    # Normalize the histogram
    hist = x[:, 1]/sum(x[:, 1])
    lbp_entropy = -sum([p * math.log(p, 2) for p in hist if p != 0])
    return lbp_entropy


#############################################################################
##                                                                         ##
##                                                                         ##
## Which : Entropy of Color histogram(Color domain)
## What  : Representation of distribution of colors in an image(RGB)
## Why   : Genuine images are supposed to have more information contained in
##         colors as  they would have been captured by camera
## How   : Computing the number of pixels that have colors in each of a
##         fixed list of color ranges
##                                                                         ##
##                                                                         ##
#############################################################################
def calc_entropy_color_hist(img):
    cal_hist = cv2.calcHist([img], [0, 1, 2],None, [10, 10, 10], [0, 256, 0, 256, 0, 256])
    # Calculate the histogram
    y = itemfreq(cal_hist.ravel())
    # Normalize the histogram
    hist_color = y[:, 1]/sum(y[:, 1])
    # entropy
    entropy_color = -sum([p * math.log(p, 2) for p in hist_color if p != 0])
    return entropy_color


############################################################################
##                                                                         ##
##                                                                         ##
## Which : Entropy of Oriented Gradient Histogram(Color domain)
## What  : Representation of distribution of colors in an image(RGB)
## Why   : Genuine images are supposed to have more information contained in
##         orientations
## How   : Compute the edge orientations
##                                                                         ##
##                                                                         ##
#############################################################################

def calc_entropy_hog(img):
    img_gray = rgb2gray(img)
    fd, hog_image =hog(image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True, multichannel=True)
    # Calculate the histogram
    x = itemfreq(hog_image.ravel())
    # Normalize the histogram
    hist = x[:, 1]/sum(x[:, 1])
    entropy_hog = -sum([p * math.log(p, 2) for p in hist if p != 0])
    return entropy_hog

#############################################################################
##                                                                         ##
##                                                                         ##
## Which : Mean,Variance,Kurtosis,Skewness of RGB channels
## What  : Representation of distribution of colors in an image(RGB)
## Why   : Genuine images are supposed to have more information contained in
##         colors as  they would have been captured by camera
## How   : Computing the number of pixels that have colors in each of a
##         fixed list of color ranges
##                                                                         ##
##                                                                         ##
#############################################################################
def calc_channel_hist(img):
    chans = cv2.split(img)
    #print(chans)
    mean=[]
    kurtosis=[]
    variance=[]
    skew=[]
    for chan in chans:
        # Calculate the histogram
        histo = cv2.calcHist([chan],[0],None,[100],[0,256])
        # Normalize the histogram
        hist_length = sum(histo)
        hist = [float(h) / hist_length for h in histo]
        #print(ss.describe(hist))
        skew.append(sp.skew(hist)[0])
        kurtosis.append(sp.kurtosis(hist)[0])
        mean.append(sp.tmean(hist))
        variance.append(sp.tvar(hist))
    return mean,variance,kurtosis,skew



# opening csv file to write the output
c = csv.writer(open("spam_ds3.csv",'w',encoding='utf8',newline=''))

# title line for the csv file
c.writerow(('Image_name','compression_ratio','aspect ratio','edge count','Avg edge length','SNR','Entropy of noise','Entropy of LBP','Entropy of color histogram','Entropy of HOG','Mean1','Mean2','Mean3','Variance1','Variance2','Variance3','Skew1','Skew2','Skew3','kurtosis1','kurtosis2','kurtosis3'))
#c.writerow(('Image_name','SNR','Entropy of noise','Entropy of LBP','Entropy of HOG','Variance1','Variance2','Variance3'))


# image folder path
path1 = "/gdrive/My Drive/Colab Notebooks/data2"
#path1 = "dredze_spam/"

#process n images simulateously to decrease the processing time
#p = Pool(10)
# Mapping to the extract_feature function with listing
#p.map(extract_feature, listing)

print()
print( "Extracting features for following images.... ")

# check the start time
start=time.time()


spamData=[]
label=[]
hashList=[]
count=-1
cnt=1
cnt2=1
for folder in os.listdir(DATA_PATH):
    print(folder)
   
    if folder=="spam":
      cnt+=1    
      print(">>>Reading ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~",folder)
      for file in os.listdir(DATA_PATH+'/'+folder):
      #print(0)
        if(str(file).endswith('.jpg') or str(file).endswith('.JPG') or str(file).endswith('.jpeg') or str(file).endswith('.JPEG')):
          cnt+=1
          #print(file, cnt)
        try:  
          img = cv2.imread(DATA_PATH+'/'+folder+'/'+file)
          #print(DATA_PATH+'/'+folder+'/'+file)
        #image = Image.open(DATA_PATH+'/'+folder+'/'+file)
          image=Image.open(DATA_PATH+'/'+folder+'/'+file)
          print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2')
        
          #print(img, image,file)      
        
         
          # compression ratio
          compression_ratio = calc_compression_ratio(img,image,file)
            # aspect ratio
          aspect_ratio = calc_aspect_ratio(img)
            # number of edges
          edge_count = calc_edge_count(img)
            # Average length of edges
          avg_edge_len = calc_avg_edge_length(img)
            # signal to noise ratio
          snr = calc_snr(img)
            # Entropy of noise
          noise_entropy = calc_entropy_noise(img)
            # Entropy of LBP histogram
          lbp_entropy = calc_entropy_lbp(img)
            # Entropy of color histogram
          entropy_color = calc_entropy_color_hist(img)
            # Entropy of Oriented Gradient Histogram (HOG)
          entropy_hog = calc_entropy_hog(img)
            # Mean,variance,kurtosis,skewness of RGB channels
          mean,var,kurt,skew = calc_channel_hist(img)
            # writing data to csv file
          
          c.writerow((file,compression_ratio,aspect_ratio,edge_count,avg_edge_len,snr,noise_entropy,lbp_entropy,entropy_color,entropy_hog,mean[0],mean[1],mean[2],var[0],var[1],var[2],skew[0],skew[1],skew[2],kurt[0],kurt[1],kurt[2]))

        except:
          cnt2+=1
          print('cnt2: ', cnt2)
          continue

#check the end time
end=time.time()

#print()
print("Done ... writing file ")
#print()
print('Time taken in seconds:',end-start)
          
          

'''
          try:
            img = cv2.imread(DATA_PATH+'/'+folder+'/'+file)
            image = Image.open(DATA_PATH+'/'+folder+'/'+file)
            print(2)
            print(img)
            print(image)
            # compression ratio
            compression_ratio = calc_compression_ratio(img,image,file)
            # aspect ratio
            aspect_ratio = calc_aspect_ratio(img)
            # number of edges
            edge_count = calc_edge_count(img)
            # Average length of edges
            avg_edge_len = calc_avg_edge_length(img)
            # signal to noise ratio
            snr = calc_snr(img)
            # Entropy of noise
            noise_entropy = calc_entropy_noise(img)
            # Entropy of LBP histogram
            lbp_entropy = calc_entropy_lbp(img)
            # Entropy of color histogram
            entropy_color = calc_entropy_color_hist(img)
            # Entropy of Oriented Gradient Histogram (HOG)
            entropy_hog = calc_entropy_hog(img)
            # Mean,variance,kurtosis,skewness of RGB channels
            mean,var,kurt,skew = calc_channel_hist(img)
            # writing data to csv file
            c.writerow((file,compression_ratio,aspect_ratio,edge_count,avg_edge_len,snr,noise_entropy,lbp_entropy,entropy_color,entropy_hog,mean[0],mean[1],mean[2],var[0],var[1],var[2],skew[0],skew[1],skew[2],kurt[0],kurt[1],kurt[2]))
          except:
            print(1)
            continue

#check the end time
end=time.time()

print()
print("Done ... writing file ")
print()
print('Time taken in seconds:',end-start)
'''


Extracting features for following images.... 
ham
spam
>>>Reading ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ spam
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:187: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:209: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:233: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  3
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  4
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  5
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  6
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  7
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:103: RuntimeWarning: invalid value encountered in long_scalars


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  10220
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  10221
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  10222
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  10223
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  10224
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  10225
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  10226
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  10227
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2: 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:126: RuntimeWarning: divide by zero encountered in double_scalars


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  12484
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  12485
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  12486
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  12487
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  12488
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  12489
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  12490
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~2
cnt2:  12491
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

'\n          try:\n            img = cv2.imread(DATA_PATH+\'/\'+folder+\'/\'+file)\n            image = Image.open(DATA_PATH+\'/\'+folder+\'/\'+file)\n            print(2)\n            print(img)\n            print(image)\n            # compression ratio\n            compression_ratio = calc_compression_ratio(img,image,file)\n            # aspect ratio\n            aspect_ratio = calc_aspect_ratio(img)\n            # number of edges\n            edge_count = calc_edge_count(img)\n            # Average length of edges\n            avg_edge_len = calc_avg_edge_length(img)\n            # signal to noise ratio\n            snr = calc_snr(img)\n            # Entropy of noise\n            noise_entropy = calc_entropy_noise(img)\n            # Entropy of LBP histogram\n            lbp_entropy = calc_entropy_lbp(img)\n            # Entropy of color histogram\n            entropy_color = calc_entropy_color_hist(img)\n            # Entropy of Oriented Gradient Histogram (HOG)\n            entrop

In [0]:
__author__ = 'annapurnaannadatha'

#############################################################################
##                               README                                    ##
##                                                                         ##
##     This program classifies SPAM and HAM images using SVM (Support      ##
##     Vector machine) technique. The image features values for the        ##
##     malware and benign are used to train and test the data. It takes    ##
##     image feature values for two classes( ham.csv, spam.csv). These     ##
##     files are output of the feature extraction module.                  ##
##                                                                         ##
##                                                                         ##
#############################################################################

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn import svm, preprocessing
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix
from matplotlib import style
style.use("ggplot")
import warnings  # to ignore all the deprecated warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


def main():
   # listing the features included
   data_headers = ['compression_ratio','aspect ratio','edge count','Avg edge length','SNR','Entropy of noise','Entropy of LBP','Entropy of color histogram','Entropy of HOG','Mean1','Mean2','Mean3','Variance1','Variance2','Variance3','Skew1','Skew2','Skew3','kurtosis1','kurtosis2','kurtosis3']

   # Reading malware data from csv file into an array
   data_df = pd.read_csv("spam_ds3.csv")
   malware_X = np.array(data_df[data_headers].values)
   print("No of malware files with no of features:",malware_X.shape)

   # classifying malware data as 1
   test_mal_category = np.empty([len(malware_X),1])

   for i in range(0,len(malware_X)):
       test_mal_category[i]= 1
 
   #Reading benign data from csv file into an array
   data1_df = pd.read_csv("ham_ds3.csv")
   benign_X = np.array(data1_df[data_headers].values)
   print("No of benign files with no of features:",benign_X.shape)

   # classifying benign data as -1
   test_ben_category = np.empty([len(benign_X),1])
   for i in range(0,len(benign_X)):
       test_ben_category[i]= -1

   # test size is set as 100 for each of benign and malware, so total test set include 200 files
   test_size = 400

   # merging benign and malware data into test_set and train_set
   test_set = np.concatenate((malware_X[:test_size],benign_X[:test_size]))
   train_set = np.concatenate((malware_X[test_size:],benign_X[test_size:]))

   print("train set size with number of features:",train_set.shape)
   print("test set size with number of features:",test_set.shape)

   # merging the out(classification) data for test set and train set
   test_out = np.concatenate((test_mal_category[:test_size],test_ben_category[:test_size]))
   train_out = np.concatenate((test_mal_category[test_size:],test_ben_category[test_size:]))

   # preprocessing and scaling of the data
   train_set = preprocessing.scale(train_set) 
   test_set = preprocessing.scale(test_set)

   print("######################################################################")
   
   #test_set = np.array([i for i in test_set if i])

   print(train_out.shape)
   print(train_set.shape)
   print(test_out.shape)
   print(test_set.shape)
   print('-----------------------------')

   test_out=test_out[~np.isnan(test_out).any(axis=1)]  
   train_out=train_out[~np.isnan(train_out).any(axis=1)]  
   train_set=train_set[~np.isnan(train_set).any(axis=1)]
   test_set=test_set[~np.isnan(test_set).any(axis=1)]  
   #print(train_set)

   
   print(train_out.shape)
   print(train_set.shape)
   print(test_out.shape)
   print(test_set.shape)
   print('-----------------------------')
   

   print(train_out)
   #b = np.array([[1.]])
   
  # train_set = np.concatenate((train_set, b))
   
   
   #c = np.array([[1.]])
   #test_set = np.concatenate((test_set, c))
   


   for i in range(0,81):
      #print(1)
      train_out = np.delete(train_out, (i), axis=0) 
   
   for i in range(0,33):
      #print(1)
      test_out = np.delete(test_out, (i), axis=0) 

   print(train_out.shape)
   print(train_set.shape)
   print(test_out.shape)
   print(test_set.shape)

   #c = csv.writer(open("train_set.csv",'w',encoding='utf8',newline=''))
   #c2 = csv.writer(open("train_out.csv",'w',encoding='utf8',newline=''))
   # title line for the csv file
   #c.writerow(train_set)
   #c2.writerow(train_out)


   ## classification
   classification("rbf",test_set,train_set,test_out,train_out)
   classification("linear",test_set,train_set,test_out,train_out)
   classification("poly",test_set,train_set,test_out,train_out)
  
   #svm_weights = compute_svm_weights(train_set,train_out)
   #plot_hist_weights(train_set,svm_weights)

   ## RFE feature selection with number of features to be selected
   i = 12
   ## RFE classification
   rfe_classification("rbf",i,test_set,train_set,test_out,train_out)
   rfe_classification("linear",i,test_set,train_set,test_out,train_out)
   rfe_classification("poly",i,test_set,train_set,test_out,train_out)






## SVM classification
def classification(kernel,test_set,train_set,test_out,train_out):
    #kernel
    clf = svm.SVC(kernel= kernel, C= 1.00)
    print("---------------------"  + kernel +" Kernel----------------------")
    print(train_set)
    clf_model= clf.fit(train_set,train_out.ravel())
    predicted = clf.predict(test_set)
    print_evaluation(predicted,test_out)
    ## ROC curveeee
    #compute_roc_curve(kernel,21,clf_model,test_set,train_set,test_out,train_out)
    print("######################################################################")

## To print roc curve  points
def roc_points_print(fpr,tpr):
    for i in range(len(fpr)):
        print(fpr[i],"\t",tpr[i])

## Plots roc curve and prints AUC value
def compute_roc_curve(kernel,i,model,test_set,train_set,test_out,train_out):
    # To compute and plot ROC curve
    y_score = model.decision_function(test_set)
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    fpr, tpr, _ = roc_curve(test_out, y_score)
    roc_auc = auc(fpr, tpr)
    print("AUC = %0.2f"%roc_auc)
    # Plot of a ROC curve for a specific class
    plt.figure()
    plt.plot(fpr, tpr,color ='b', label='AUC with %d features = %0.2f'%( i,roc_auc))
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(kernel)
    plt.legend(loc="lower right")
    plt.show()
    #print
    #roc_points_print(fpr,tpr)

## Computes Linear SVM weights 
def compute_svm_weights(train_set,train_out):
    # svm weights for linear kernel only
    #print(1)
    clf = svm.SVC(kernel="linear", C= 1.00)
    clf.fit(train_set,train_out.ravel())
    svm_weights = (clf.coef_ ** 2).sum(axis=0)
    svm_weights /= svm_weights.max()
    print(svm_weights)
    return svm_weights

## PLots histogram of the SVM wieghts
def plot_hist_weights(train_set,svm_weights):
    X_indices = np.arange(train_set.shape[-1])
    #  plotting histogram for the svm weights of each feature
    plt.bar(X_indices - .45, svm_weights, width=0.5, label='SVM weight', color='r')
    plt.title("SVM weights")
    plt.xlabel('Feature')
    plt.yticks(())
    plt.xticks(X_indices,('comp','aspect','edges','edgelen','SNR','noise','LBP','color_hist','HOG','Mean1','Mean2','Mean3','Var1','Var2','Vari3','Skew1','Skew2','Skew3','kurt1','kurt2','kurt3'))
    plt.axis('tight')
    plt.legend(loc='upper right')
    plt.show()

    # plotting rfe algorithm ranks on the same histogram of svm weights for comparision####################
    #plt.bar(X_indices - .05, rfe.support_, width=.5, label='RFE Selection', color='b')
    #plt.legend(loc='upper right')
    #plt.axis('tight')
    #plt.show()############


## RFE selection and classification based no features selected
def rfe_classification(kernel,i,test_set,train_set,test_out,train_out):
    # Recursive feature elimation
    model = svm.SVC(kernel="linear", C=1.00) #LogisticRegression()
    rfe = RFE(model,i) # no of features to be selected
    rfe = rfe.fit(train_set,train_out.ravel())
    rfe_weights = (rfe.support_ ** 2).sum(axis=0)
    rfe_weights /= rfe_weights.max()
    # Transform the featureset from 21 to subset (i)  as selected in the RFE computation above
    rfe_train = rfe.transform(train_set)
    rfe_test = rfe.transform(test_set)
    clf = svm.SVC(kernel=kernel, C= 1.00)
    print("---------------------RFE : "+kernel+" Kernel----------------------")
    rfe_model = clf.fit(rfe_train,train_out.ravel())
    rbf_predict = clf.predict(rfe_test)
    print_evaluation(rbf_predict,test_out)
    #roc
    #compute_roc_curve(kernel,i,rfe_model,rfe_test,rfe_train,test_out,train_out)
    print("######################################################################")


## Prints the final evaluation
def print_evaluation(predict,test_out):
    conf_mat = confusion_matrix(test_out,predict)
    print(conf_mat)
    TN = conf_mat[0][0]
    FP = conf_mat[0][1]
    FN = conf_mat[1][0]
    TP = conf_mat[1][1]
    Accuracy = (TP+TN)/(TP+FP+FN+TN)
    print("Accuracy= %0.2f"%Accuracy)
    TPR = TP/(TP + FN)
    print("Recall TPR=%0.2f"%TPR)
    FPR = FP/(FP + TN)
    print("FPR=%0.2f"%FPR)
    PPR = TP/(TP +FP)
    print("Precision PPR=%0.2f"%PPR)



## Main
if __name__ == "__main__":
    main()



No of malware files with no of features: (2022, 21)
No of benign files with no of features: (1198, 21)
train set size with number of features: (2420, 21)
test set size with number of features: (800, 21)
######################################################################
(2420, 1)
(2420, 21)
(800, 1)
(800, 21)
-----------------------------
(2420, 1)
(2339, 21)
(800, 1)
(767, 21)
-----------------------------
[[ 1.]
 [ 1.]
 [ 1.]
 ...
 [-1.]
 [-1.]
 [-1.]]
(2339, 1)
(2339, 21)
(767, 1)
(767, 21)
---------------------rbf Kernel----------------------
[[-0.35232694 -0.31033622 -0.14473179 ... -1.21796606 -1.12558981
  -1.16538764]
 [ 0.80659259  0.10305705 -0.07336393 ...  0.99371004  0.99850644
   1.01313864]
 [-0.60037568  0.22902556 -0.13775554 ...  0.87037082  0.24803175
   0.75633674]
 ...
 [-1.19306576  0.11563692  0.43604128 ...  0.83671819 -0.23891616
   0.09527282]
 [-0.07606233 -0.39515023 -0.04553819 ... -1.40685656 -1.50151548
  -1.59003756]
 [-0.31524326 -0.15277912  3.06555

In [0]:
from google.colab import drive
drive.mount('/content/drive')